In [8]:
"""
Challenge #3
앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

Hint.
예시 제공
요구조건에 맞는 답변 형식을 생성하도록 적절한 예시를 만들고, FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate를 이용하여 LLM에게 예시를 제공하세요.
자세한 사용법은 다음 공식 문서를 참고해보세요
Few-shot prompt templates
Few-shot examples for chat models
메모리 활용
ConversationBufferMemory 등 강의에서 배운 메모리 중 하나를 사용하여 이전 대화 기록을 기억하고 기록을 이용한 답변을 제공할 수 있도록 합니다.
채팅 형식의 메모리 기록을 프롬프트에 추가하고 싶을 때는 MessagesPlaceholder를 이용하세요. (공식문서 예시)
RunnablePassthrough를 활용하면 LCEL 체인을 구현할 때 메모리 적용을 쉽게 할 수 있습니다. RunnablePassthrough는 메모리를 포함한 데이터를 체인의 각 단계에 전달하는 역할을 합니다. (강의 #5.7 1:04~ 참고)
"""
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature= 0.1,
    streaming=False,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [9]:
examples = [
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
    {"input": "어벤져스", "output": "🛡️🧙‍♂️🧟‍♂️"},
]


In [10]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="영화 제목: {input}\n이모티콘: {output}",
)

In [11]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 영화 제목을 세 개의 이모티콘으로 나타내세요.",
    suffix="영화 제목: {movie_title}\n이모티콘:",
    input_variables=["movie_title"],
)

In [12]:
memory = ConversationBufferMemory(return_messages=True)


In [13]:
lcel_chain = LLMChain(
    llm=llm,
    prompt=few_shot_prompt,
    memory=memory,
)

In [14]:
response1 = lcel_chain.run(movie_title="탑건")
print("첫 번째 응답:", response1)

첫 번째 응답: 🛩️👨‍✈️🔥


In [15]:
response2 = lcel_chain.run(movie_title="아바타")
print("두 번째 응답:", response2)

두 번째 응답: 🌿🌎🔵


In [16]:
previous_question = memory.chat_memory.messages[-2]  # 마지막 HumanMessage
print("이전 질문:", previous_question.content)

이전 질문: 아바타


In [18]:
previous_question = memory.chat_memory.messages[-3]  # 마지막 HumanMessage
print("이전 질문:", previous_question.content)

이전 질문: 🛩️👨‍✈️🔥


In [19]:
previous_question = memory.chat_memory.messages[-4]  # 마지막 HumanMessage
print("이전 질문:", previous_question.content)#

이전 질문: 탑건
